## 2.2) Setting up the foreground system: foreground database

This notebook contains the required steps to import the foreground datatabase. The database comes originally as an excel file.

In [1]:
from pathlib import Path
from pprint import pprint

In [2]:
## Import BW25 packages
import bw2data as bd
import bw2io as bi

In [3]:
#Importing the variables with the project name and background db
from project_details import ei_name, project_name

In [4]:
bd.projects.set_current(project_name)
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-391-cutoff

In [5]:
# Is the background database name the same as the one we wrote in `project_details.py`?
assert ei_name in bd.databases

## Transform the data from xlsx format to Brightway format

### Extract foreground inventory data

In [9]:
# Include the path to the foreground database (excel file)
FG_DB = Path("./Foreground.xlsx")

# Import your LCI
lci = bi.ExcelImporter(FG_DB)

Extracted 1 worksheets in 0.06 seconds


### Match the foreground database to itself

In [10]:
# Need to match FG_DB to itself
lci.match_database(fields=["name", "unit", "location"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the background db

In [11]:
# Need to match FG_DB to the biosphere
lci.match_database(ei_name, fields=["name", "reference product", "location", "unit"])

Applying strategy: link_iterable_by_fields


### Match the FG_DB to the biosphere3 database

In [12]:
# Need to match FG_DB to the biosphere
lci.match_database("biosphere3", fields=["name", "categories"])

Applying strategy: link_iterable_by_fields


In [14]:
bi.create_core_migrations()
# Once your package is imported we need to apply strategies
lci.apply_strategies()

# We need to match databases - name and categories but ATTENTION! the categories in
# the excel file is "None" so we will lci.write_excel() also need to match against unit.

lci.statistics()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 4.86 seconds
26 datasets
123 exchanges
0 unlinked exchanges
  


(26, 123, 0)

In [15]:
import_statistics = lci.statistics(print_stats=False)

# If all exchanges have been linked, proceed to write the database
if import_statistics[2] == 0:
    lci.write_database()
else:
    print(f"There are unlinked exchanges {import_statistics}")
    lci.write_excel()
    for unlinked_exchange in lci.unlinked:
        pprint(unlinked_exchange)

bd.databases

100%|████████████████████████████████████████| 26/26 [00:00<00:00, 17983.49it/s]


Vacuuming database 
Created database: asphalt


Databases dictionary with 3 object(s):
	asphalt
	biosphere3
	ecoinvent-391-cutoff

### DB structure checks

In [16]:
import bw2analyzer as bwa

Let's define and assign the name of the foreground database to a variable called "fg_name"
The name must be equal to the content of the cell B2 in the excel file "Foreground.xlsx"

In [24]:
fg_name = "asphalt"

There are many properties and functions associated with this database object. To access those, let's assign the database to a variable

In [25]:
fgdb = bd.Database(fg_name)

Let's explore the foreground db with the function "random"

In [27]:
fgdb.random()

'other fillers, production and transport' (kilogram, NL, None)

In [28]:
random_fg_activity = fgdb.random()
random_fg_activity

'asphalt, transport to processing' (kilogram, NL, None)

We can get the type of the object that was returned from the database:

In [29]:
type(random_fg_activity)

bw2data.backends.proxies.Activity

The type is an activity proxy. Activity proxies allow us to interact with the content of the database.

To see what the activity contains, we can convert it to a dictionary:

In [30]:
random_fg_activity.as_dict()

{'reference product': 'asphalt',
 'code': 'asphalt_C2',
 'location': 'NL',
 'amount': 1,
 'unit': 'kilogram',
 'original_ConversionDem2FU': 1,
 'name': 'asphalt, transport to processing',
 'worksheet name': 'Foreground',
 'database': 'asphalt',
 'id': 25969}

We can get the value of each item by specifying its key. Example: let's get the value corresponding to the key 'location'

In [32]:
random_fg_activity['location']

'NL'

<span style='background:yellow'> Ended here </span> 

In [20]:
pavement = fgdb.get('asphalt_SMA')

In [21]:
pavement

'SMA' (kilogram, NL, None)

In [22]:
bwa.print_recursive_supply_chain(pavement, max_level=1)

1: 'SMA' (kilogram, NL, None)
  1: 'SMA, materials and transport to plant' (kilogram, NL, None)
  1: 'SMA, production' (kilogram, NL, None)
  1: 'asphalt, transport to site' (kilogram, NL, None)
  1: 'asphalt, construction' (kilogram, NL, None)
  1: 'asphalt, pvi, car' (kilometer, NL, None)
  1: 'asphalt, pvi, HDV' (kilometer, NL, None)
  1: 'asphalt, pvi, trailer' (kilometer, NL, None)
  1: 'asphalt, demolition' (kilogram, NL, None)
  1: 'asphalt, transport to processing' (kilogram, NL, None)
  1: 'asphalt, processing' (kilogram, NL, None)


In [23]:
bwa.print_recursive_supply_chain(pavement, max_level=2)

1: 'SMA' (kilogram, NL, None)
  1: 'SMA, materials and transport to plant' (kilogram, NL, None)
    0.068: 'bitumen, production and transport' (kilogram, NL, None)
    0.676: 'crushed stone, production and transport' (kilogram, NL, None)
    0.075: 'crushed sand, production and transport' (kilogram, NL, None)
    0.091: 'other fillers, production and transport' (kilogram, NL, None)
    0.073: 'natural sand, production and transport' (kilogram, NL, None)
    0.014: 'weak filler, production and transport' (kilogram, NL, None)
    0.003: 'drip resistant material, production and transport' (kilogram, NL, None)
  1: 'SMA, production' (kilogram, NL, None)
    0.00714: 'market for electricity, low voltage' (kilowatt hour, NL, None)
    0.254: 'heat production, natural gas, at industrial furnace >100kW' (megajoule, Europe without Switzerland, None)
    0.0043: 'diesel, burned in building machine' (megajoule, GLO, None)
  1: 'asphalt, transport to site' (kilogram, NL, None)
    0.0333: 'transpo